In [34]:
import numpy as np
import scipy.linalg
import operator
from itertools import permutations

## 1. MP Representation

In [2]:
class MarkovProcess(object):
    
    def __init__(self,transition_proba):
        """
        Initialize the MarkovProcess instance.
 
        Parameters
        ----------
        transition_proba: dictionary of dictionary
            {state : {state : probability}} representing the probabilities
            of transitioning from one state to another in the Markov 
            Process.
        
        Three attributes
        ----------
        transition_proba: dictionary of dictionary
        
        states: vector
            a vector to represent the state space of the Markov Process
        
        stationary_dist: dictionary of float
            {state: proba} to represent the long-run probability of being in each state
        """
        self.transition_proba = transition_proba
        self.states = list(transition_proba.keys())
        self.stationary_dist = None
    
    def calculate_stationary_distribution(self):
        # probability dictionary to matrix
        num = len(self.states)
        T = np.empty([num,num])
        i = 0
        for state, proba in self.transition_proba.items():
            T[i,:] = list(proba.values())
            i = i+1
        """
        Stationary distribution is given by solving
        pi = pi*T.
        Solve for the left (row) eigen vector of transition matrix
        """      
        value, left, right = scipy.linalg.eig(T, left=True,right=True)
        eig_vec = left[:,np.where(np.abs(value-1)<1e-8)[0][0]].astype(float)
        eig_vec = eig_vec/np.sum(eig_vec) # normalize
        self.stationary_dist = {}
        for i in range(num):
            self.stationary_dist[self.states[i]] = eig_vec[i]
        

Example:

In [3]:
T_0 = {'Facebook':{'Facebook':0.9,'Class 1':0.1,'Class 2':0,'Class 3':0,'Pass':0,'Sleep':0,'Pub':0},
     'Class 1':{'Facebook':0.5,'Class 1':0,'Class 2':0.5,'Class 3':0,'Pass':0,'Sleep':0,'Pub':0},
     'Class 2':{'Facebook':0,'Class 1':0,'Class 2':0,'Class 3':0.8,'Pass':0,'Sleep':0.2,'Pub':0},
     'Class 3':{'Facebook':0,'Class 1':0,'Class 2':0,'Class 3':0,'Pass':0.6,'Sleep':0,'Pub':0.4},
     'Pass':{'Facebook':0,'Class 1':0,'Class 2':0,'Class 3':0,'Pass':0,'Sleep':1,'Pub':0},
     'Sleep':{'Facebook':0,'Class 1':0,'Class 2':0,'Class 3':0,'Pass':0,'Sleep':1,'Pub':0},
     'Pub':{'Facebook':0,'Class 1':0.2,'Class 2':0.4,'Class 3':0.4,'Pass':0,'Sleep':0,'Pub':0}}

In [4]:
mp = MarkovProcess(T_0)
print('The state space of the student MP problem:')
print(mp.states)
print('\n')
print('The stationary distribution of the student MP problem:')
mp.calculate_stationary_distribution()
print(mp.stationary_dist)

The state space of the student MP problem:
['Facebook', 'Class 1', 'Class 2', 'Class 3', 'Pass', 'Sleep', 'Pub']


The stationary distribution of the student MP problem:
{'Facebook': 0.0, 'Class 1': 0.0, 'Class 2': 0.0, 'Class 3': 0.0, 'Pass': 0.0, 'Sleep': 1.0, 'Pub': 0.0}


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: ComplexWarning: Casting complex values to real discards the imaginary part


## 2. MRP Representation 

In [5]:
class MarkovRewardProcess(object):
    
    def __init__(self,transition_proba,reward,gamma):
        """
        Initialize the MarkovRewardProcess instance.
 
        Parameters
        ----------
        transition_proba: dictionary of dictionary
            {state : {state : probability}} representing the probabilities
            of transitioning from one state to another in the Markov 
            (Reward) Process.
        
        reward: dictionary of dictionary
            {state : {state : reward}} representing the immediate reward
            of transitioning from one state to another in the Markov 
            Reward Process.
        
        gamma: float
            a float between 0 and 1 to discount long-run rewards
            
        Five attributes
        ----------
        transition_proba: dictionary of dictionary
        
        states: vector
            a vector to represent the state space of the Markov Process
        
        reward: dictionary of dictionary
        
        gamma: float
        
        values: dictionary of float
            {state: value} to represent the optimal value function of each state
        """
        self.transition_proba = transition_proba
        self.states = list(transition_proba.keys())
        self.reward = reward
        self.gamma = gamma
        
        self.values = None
    
    def is_terminal(self):
        """
        Check whether each state is self-absorbing
        
        return a dictionary of bools
        """
        is_terminal_dict = {s: t[s] == 1 for s,t in self.transition_proba.items()}
        return is_terminal_dict
            
    def get_transition_matrix(self):
        """
        A helper function to transform the transition dictionary to a 2-D matrix
        """
        num = len(self.states)
        T = np.empty([num,num])
        for i in range(num):
            for j in range(num):
                T[i,j] = self.transition_proba[self.states[i]][self.states[j]]
        return T
    
    def get_expected_reward(self):
        """
        Transform the r(s,s') representation to R(s) = E[r(s,s')] representation
        Return a dictionary {state: reward}
        """
        T = self.get_transition_matrix()
        num = len(self.states)
        R = np.empty([num,num])
        for i in range(num):
            for j in range(num):
                R[i,j] = self.reward[self.states[i]][self.states[j]]        
        R_vec = np.diagonal(np.dot(T,R.T))
        R_dict = {}
        for i in range(num):
            R_dict[self.states[i]] = R_vec[i]
        return R_dict
    
    def get_value_function(self):
        """
        Solve Bellman Equations 
            u = R + T*u
        s.t. the terminal conditions
        
        Return a dictionary {state:value}
        """
        R_dict = self.get_expected_reward()
        is_terminal_dict = self.is_terminal()
        R = np.empty(len(self.states))
        is_terminal = list()
        for i in range(len(self.states)):
            R[i] = R_dict[self.states[i]]
            is_terminal.append(is_terminal_dict[self.states[i]])
        is_terminal = np.array(is_terminal)
        T = self.get_transition_matrix()
        v_vec = np.empty(len(self.states))
        # terminal states
        v_vec[is_terminal] = R[is_terminal]
        # solve bellman equations for non-terminal states
        v_recur = v_vec[~is_terminal]
        R_inter = R[~is_terminal]+ np.dot(T[:,is_terminal][~is_terminal],v_vec[is_terminal])
        v_recur = np.linalg.solve(self.gamma*T[:,~is_terminal][~is_terminal]-np.eye(T[:,~is_terminal][~is_terminal].shape[0]),-R_inter)
        v_vec[~is_terminal] = v_recur
        self.values={}
        for i in range(len(self.states)):
            self.values[self.states[i]] = v_vec[i]
        return self.values

Example:

In [6]:
R_0 = {'Facebook':{'Facebook':-1,'Class 1':-1,'Class 2':0,'Class 3':0,'Pass':0,'Sleep':0,'Pub':0},
     'Class 1':{'Facebook':-2,'Class 1':0,'Class 2':-2,'Class 3':0,'Pass':0,'Sleep':0,'Pub':0},
     'Class 2':{'Facebook':0,'Class 1':0,'Class 2':0,'Class 3':-2,'Pass':0,'Sleep':-2,'Pub':0},
     'Class 3':{'Facebook':0,'Class 1':0,'Class 2':0,'Class 3':0,'Pass':-2,'Sleep':0,'Pub':-2},
     'Pass':{'Facebook':0,'Class 1':0,'Class 2':0,'Class 3':0,'Pass':0,'Sleep':10,'Pub':0},
     'Sleep':{'Facebook':0,'Class 1':0,'Class 2':0,'Class 3':0,'Pass':0,'Sleep':0,'Pub':0},
     'Pub':{'Facebook':0,'Class 1':1,'Class 2':1,'Class 3':1,'Pass':0,'Sleep':0,'Pub':0}}

In [7]:
my_mrp = MarkovRewardProcess(T_0,R_0,0.9)
print('The expectation of immediate reward of each state is')
print(my_mrp.get_expected_reward())
print('\n')
print('The long-run value of each state is')
print(my_mrp.get_value_function())

The expectation of immediate reward of each state is
{'Facebook': -1.0, 'Class 1': -2.0, 'Class 2': -2.0, 'Class 3': -2.0, 'Pass': 10.0, 'Sleep': 0.0, 'Pub': 1.0}


The long-run value of each state is
{'Facebook': -7.637608431059512, 'Class 1': -5.0127289100145225, 'Class 2': 0.9426552976939073, 'Class 3': 4.087021246797093, 'Pass': 10.0, 'Sleep': 0.0, 'Pub': 1.9083923522141464}


## 3. MDP Representation

Example: smoov & curly's bogus journey

https://www.youtube.com/playlist?list=PLw0qyFP7t4IYuI9XcDIGsARfO1QKMrpbN

In [8]:
T = {}
R = {}
for i in range(4):
    for j in range(4):
        if not(i==1 and j == 2):
            T_state = {}
            R_state = {}
            for direction in ['Up','Right','Down','Left']:
                T_state_action = {}
                R_state_action = {}
                for m in range(4):
                    for n in range(4):
                        if not(m==1 and n == 2):
                            T_state_action[(m,n)] = 0
                            if i==3 and j == 3: R_state_action[(m,n)] = 10
                            elif i == 3 and j == 2: R_state_action[(m,n)] = -10
                            else: R_state_action[(m,n)] = -1
                T_state[direction] = T_state_action
                R_state[direction] = R_state_action
            T[(i,j)] = T_state
            R[(i,j)] = R_state

In [9]:
T[(0,0)]['Up'][(0,1)] = 0.8
T[(0,0)]['Up'][(1,0)] = 0.1
T[(0,0)]['Up'][(0,0)] = 0.1
T[(0,0)]['Right'][(0,1)] = 0.1
T[(0,0)]['Right'][(1,0)] = 0.8
T[(0,0)]['Right'][(0,0)] = 0.1
T[(0,0)]['Left'][(0,0)] = 0.9
T[(0,0)]['Left'][(0,1)] = 0.1
T[(0,0)]['Down'][(0,1)] = 0.1
T[(0,0)]['Down'][(0,0)] = 0.9

T[(0,1)]['Up'][(0,2)] = 0.8
T[(0,1)]['Up'][(0,1)] = 0.1
T[(0,1)]['Up'][(1,1)] = 0.1
T[(0,1)]['Right'][(1,1)] = 0.8
T[(0,1)]['Right'][(0,2)] = 0.1
T[(0,1)]['Right'][(0,0)] = 0.1
T[(0,1)]['Left'][(0,1)] = 0.8
T[(0,1)]['Left'][(0,0)] = 0.1
T[(0,1)]['Left'][(0,2)] = 0.1
T[(0,1)]['Down'][(0,0)] = 0.8
T[(0,1)]['Down'][(0,1)] = 0.1
T[(0,1)]['Down'][(1,1)] = 0.1

T[(0,2)]['Up'][(0,3)] = 0.8
T[(0,2)]['Up'][(0,2)] = 0.2
T[(0,2)]['Right'][(0,2)] = 0.8
T[(0,2)]['Right'][(0,3)] = 0.1
T[(0,2)]['Right'][(0,1)] = 0.1
T[(0,2)]['Left'][(0,2)] = 0.8
T[(0,2)]['Left'][(0,3)] = 0.1
T[(0,2)]['Left'][(0,1)] = 0.1
T[(0,2)]['Down'][(0,1)] = 0.8
T[(0,2)]['Down'][(0,2)] = 0.2

T[(0,3)]['Up'][(0,3)] = 0.9
T[(0,3)]['Up'][(1,3)] = 0.1
T[(0,3)]['Right'][(1,3)] = 0.8
T[(0,3)]['Right'][(0,3)] = 0.1
T[(0,3)]['Right'][(0,2)] = 0.1
T[(0,3)]['Left'][(0,3)] = 0.9
T[(0,3)]['Left'][(0,2)] = 0.1
T[(0,3)]['Down'][(0,2)] = 0.8
T[(0,3)]['Down'][(0,3)] = 0.1
T[(0,3)]['Down'][(1,3)] = 0.1

T[(1,0)]['Up'][(1,1)] = 0.8
T[(1,0)]['Up'][(0,0)] = 0.1
T[(1,0)]['Up'][(2,0)] = 0.1
T[(1,0)]['Right'][(2,0)] = 0.8
T[(1,0)]['Right'][(1,0)] = 0.1
T[(1,0)]['Right'][(1,1)] = 0.1
T[(1,0)]['Left'][(0,0)] = 0.8
T[(1,0)]['Left'][(1,0)] = 0.1
T[(1,0)]['Left'][(1,1)] = 0.1
T[(1,0)]['Down'][(1,0)] = 0.8
T[(1,0)]['Down'][(2,0)] = 0.1
T[(1,0)]['Down'][(0,0)] = 0.1

T[(1,1)]['Up'][(1,1)] = 0.8
T[(1,1)]['Up'][(0,1)] = 0.1
T[(1,1)]['Up'][(2,1)] = 0.1
T[(1,1)]['Right'][(2,1)] = 0.8
T[(1,1)]['Right'][(1,1)] = 0.1
T[(1,1)]['Right'][(1,0)] = 0.1
T[(1,1)]['Left'][(0,1)] = 0.8
T[(1,1)]['Left'][(1,0)] = 0.1
T[(1,1)]['Left'][(1,1)] = 0.1
T[(1,1)]['Down'][(1,0)] = 0.8
T[(1,1)]['Down'][(2,1)] = 0.1
T[(1,1)]['Down'][(0,1)] = 0.1

T[(1,3)]['Up'][(1,3)] = 0.8
T[(1,3)]['Up'][(0,3)] = 0.1
T[(1,3)]['Up'][(2,3)] = 0.1
T[(1,3)]['Right'][(2,3)] = 0.8
T[(1,3)]['Right'][(1,3)] = 0.2
T[(1,3)]['Left'][(0,3)] = 0.8
T[(1,3)]['Left'][(1,3)] = 0.2
T[(1,3)]['Down'][(1,3)] = 0.8
T[(1,3)]['Down'][(0,3)] = 0.1
T[(1,3)]['Down'][(2,3)] = 0.1

T[(2,0)]['Up'][(2,1)] = 0.8
T[(2,0)]['Up'][(1,0)] = 0.1
T[(2,0)]['Up'][(3,0)] = 0.1
T[(2,0)]['Right'][(3,0)] = 0.8
T[(2,0)]['Right'][(2,0)] = 0.1
T[(2,0)]['Right'][(2,1)] = 0.1
T[(2,0)]['Left'][(1,0)] = 0.8
T[(2,0)]['Left'][(2,0)] = 0.1
T[(2,0)]['Left'][(2,1)] = 0.1
T[(2,0)]['Down'][(1,0)] = 0.1
T[(2,0)]['Down'][(2,0)] = 0.8
T[(2,0)]['Down'][(3,0)] = 0.1

T[(2,1)]['Up'][(2,2)] = 0.8
T[(2,1)]['Up'][(1,1)] = 0.1
T[(2,1)]['Up'][(3,1)] = 0.1
T[(2,1)]['Right'][(3,1)] = 0.8
T[(2,1)]['Right'][(2,0)] = 0.1
T[(2,1)]['Right'][(2,2)] = 0.1
T[(2,1)]['Left'][(1,1)] = 0.8
T[(2,1)]['Left'][(2,0)] = 0.1
T[(2,1)]['Left'][(2,2)] = 0.1
T[(2,1)]['Down'][(1,1)] = 0.1
T[(2,1)]['Down'][(2,0)] = 0.8
T[(2,1)]['Down'][(3,1)] = 0.1

T[(2,2)]['Up'][(2,3)] = 0.8
T[(2,2)]['Up'][(2,2)] = 0.1
T[(2,2)]['Up'][(3,2)] = 0.1
T[(2,2)]['Right'][(3,1)] = 0.8
T[(2,2)]['Right'][(2,0)] = 0.1
T[(2,2)]['Right'][(2,2)] = 0.1
T[(2,2)]['Left'][(1,1)] = 0.8
T[(2,2)]['Left'][(2,0)] = 0.1
T[(2,2)]['Left'][(2,2)] = 0.1
T[(2,2)]['Down'][(2,2)] = 0.1
T[(2,2)]['Down'][(2,1)] = 0.8
T[(2,2)]['Down'][(3,2)] = 0.1

T[(2,3)]['Up'][(2,3)] = 0.8
T[(2,3)]['Up'][(1,3)] = 0.1
T[(2,3)]['Up'][(3,3)] = 0.1
T[(2,3)]['Right'][(3,3)] = 0.8
T[(2,3)]['Right'][(2,3)] = 0.1
T[(2,3)]['Right'][(2,2)] = 0.1
T[(2,3)]['Left'][(1,3)] = 0.8
T[(2,3)]['Left'][(2,2)] = 0.1
T[(2,3)]['Left'][(2,3)] = 0.1
T[(2,3)]['Down'][(1,3)] = 0.1
T[(2,3)]['Down'][(2,2)] = 0.8
T[(2,3)]['Down'][(3,3)] = 0.1

T[(3,0)]['Up'][(3,1)] = 0.8
T[(3,0)]['Up'][(2,0)] = 0.1
T[(3,0)]['Up'][(3,0)] = 0.1
T[(3,0)]['Right'][(3,0)] = 0.9
T[(3,0)]['Right'][(3,1)] = 0.1
T[(3,0)]['Left'][(2,0)] = 0.8
T[(3,0)]['Left'][(3,0)] = 0.1
T[(3,0)]['Left'][(3,1)] = 0.1
T[(3,0)]['Down'][(3,0)] = 0.9
T[(3,0)]['Down'][(2,0)] = 0.1

T[(3,1)]['Up'][(3,2)] = 0.8
T[(3,1)]['Up'][(2,1)] = 0.1
T[(3,1)]['Up'][(3,1)] = 0.1
T[(3,1)]['Right'][(3,1)] = 0.8
T[(3,1)]['Right'][(3,0)] = 0.1
T[(3,1)]['Right'][(3,2)] = 0.1
T[(3,1)]['Left'][(2,1)] = 0.8
T[(3,1)]['Left'][(3,0)] = 0.1
T[(3,1)]['Left'][(3,2)] = 0.1
T[(3,1)]['Down'][(3,0)] = 0.8
T[(3,1)]['Down'][(2,1)] = 0.1
T[(3,1)]['Down'][(3,1)] = 0.1

for direction in ['Up','Right','Down','Left']:
    T[(3,2)][direction][(3,2)] = 1
    T[(3,3)][direction][(3,3)] = 1

In [10]:
class MarkovDecisionProcess(object):
        def __init__(self,transition_proba,reward,gamma,tol):
            """
            Initialize the MarkovDecisionProcess instance.

            Parameters
            ----------
            transition_proba: dictionary of dictionary of dictionary
                {state : {action: {state : probability}}} representing the probabilities
                of transitioning from one state to another in the Markov 
                Decision Process.

            reward: dictionary of dictionary of dictionary
                {state : {action: {state : reward}}} representing the immediate reward
                of transitioning from one state to another in the Markov 
                Decision Process.

            gamma: float
                a float between 0 and 1 to discount long-run rewards
            
            tol: float
                the tolerance level of error in iteration
                
            Eight attributes
            ----------
            transition_proba

            states: vector
                a vector to represent the state space of the Markov Process
    
            actions: vector
                a vector to represent the action space of the Markov
            
            reward

            gamma
            
            tol
            
            policy: dictionary of dictionary
                {state: {action: proba}} to represent the stochastic policy
            
            values: dictionary of float
                {state: value} to represent the optimal value function of each state
            """
            self.transition_proba = transition_proba
            self.reward = reward
            self.gamma = gamma
            self.tol = tol
            self.states = list(transition_proba.keys())
            self.actions = list(transition_proba[self.states[0]].keys())

            self.policy = None
            self.values = None

        
        def dict_to_matrix_1(self,dictionary):        
            M = np.empty([len(self.states),len(self.actions),len(self.states)])
            for i in range(len(self.states)):
                for a in range(len(self.actions)):
                    for j in range(len(self.states)):
                        M[i][a][j]= dictionary[self.states[i]][self.actions[a]][self.states[j]]
            return M
        
        def dict_to_matrix_2(self,dictionary):
            M = np.empty([len(self.states),len(self.actions)])
            for i in range(len(self.states)):
                for a in range(len(self.actions)):
                    M[i][a]= dictionary[self.states[i]][self.actions[a]]
            return M
        
        def get_expected_reward(self):
            """
            Calculate the expected reward per (state,action)
            
            Return {state: {action: reward}}
            """  
            R = self.dict_to_matrix_1(self.reward)
            T = self.dict_to_matrix_1(self.transition_proba)
            R_expected_dict = {}
            for i in range(len(self.states)):
                R_state_reward_vec = np.diagonal(np.dot(T[i][:][:],(R[i][:][:]).T))
                R_state_reward = {}
                for a in range(len(self.actions)):
                    R_state_reward[self.actions[a]] = R_state_reward_vec[a]
                R_expected_dict[self.states[i]]= R_state_reward
            return R_expected_dict
        
        def to_MRP(self,policy_dict):
            """
            Given a policy, represent the problem as a Markov Reward Process
            """       
            num = len(self.states)
            
            T_dict = {}
            T_matrix = self.dict_to_matrix_1(self.transition_proba)
            for i in range(num):
                T_state_vec = np.zeros(num)
                T_state_dict = {}
                for a in range(len(self.actions)):
                    T_state_vec += policy_dict[self.states[i]][self.actions[a]]*T_matrix[i][a][:]
                for j in range(num):
                    T_state_dict[self.states[j]] = T_state_vec[j]
                T_dict[self.states[i]] = T_state_dict
            
            R_dict = {}
            R_matrix = self.dict_to_matrix_1(self.reward)
            for i in range(num):
                R_state_vec = np.zeros(num)
                R_state_dict = {}
                for a in range(len(self.actions)):
                    R_state_vec += policy_dict[self.states[i]][self.actions[a]]*R_matrix[i][a][:]
                for j in range(num):
                    R_state_dict[self.states[j]] = R_state_vec[j]
                R_dict[self.states[i]] = R_state_dict            
            
            return MarkovRewardProcess(T_dict,R_dict,self.gamma)
        
        def is_terminal(self,s,a):
            return self.transition_proba[s][a][s] == 1
        
        def get_state_value_function(self,policy_dict):
            """
            Given a policy, return the value function of the corresponding MRP problem
            
            Return {state: value}
            """                 
            mrp = self.to_MRP(policy_dict)
            values = mrp.get_value_function()
            return values
        
        def get_state_act_value_function(self,policy_dict):
            """
            Given a policy, return the action value function of the corresponding MRP problem
            
            Return {state: {action: value}}
            """ 
            v_dict = self.get_state_value_function(policy_dict)
            v_vec = np.empty(len(self.states))
            for i in range(len(self.states)):
                v_vec[i] = v_dict[self.states[i]]
                
            R_expected_dict = self.get_expected_reward()
            R_matrix = self.dict_to_matrix_2(R_expected_dict)
                    
            T = self.dict_to_matrix_1(self.transition_proba)
            
            q = {}
            q_matrix = R_matrix + np.dot(T,v_vec)
            for i in range(len(self.states)):
                q_state = {}
                for a in range(len(self.actions)):
                    q_state[self.actions[a]] = q_matrix[i][a]
                q[self.states[i]] = q_state
    
            return q
        
        def get_improved_policy(self,policy_dict):
            """
            Given a policy, find the best possible action for each state based
            on the action value function
            """ 
            q = self.get_state_act_value_function(policy_dict)
            greedy_policy = {}
            for state,action_value in q.items():
                policy_state = {}
                greedy_action = max(action_value.items(),key=operator.itemgetter(1))[0]
                for action in self.actions:
                    if action == greedy_action: policy_state[action] = 1
                    else: policy_state[action] = 0
                greedy_policy[state] = policy_state
            return greedy_policy
        
        def policy_iteration(self):
            """
            """
            # initialization
            policy_dict = {s:{a:1./len(self.actions) for a in self.actions } for s in self.states }
            v_max = self.get_state_value_function(policy_dict)
            epsilon = 1000
            while epsilon >= self.tol:
                policy_dict = self.get_improved_policy(policy_dict)
                v = self.get_state_value_function(policy_dict)
                epsilon = max(abs(v[s] - v_max[s]) for s in v) 
                v_max = v
            
            return policy_dict
        
        def value_iteration(self,synchronous):
            v = np.zeros(len(self.states))
            T = self.dict_to_matrix_1(self.transition_proba)
            R_matrix = self.dict_to_matrix_2(self.get_expected_reward())

            iterate = True           
            while iterate:
                if synchronous:
                    OldValues = v.copy()
                    v = np.max(R_matrix+self.gamma*np.dot(T,OldValues),axis=1)
            
            self.values={}
            for i in range(num_s):
                self.values[self.states[i]] = v[i]
        
        
        def generate_initial_state_action(self,policy_dict):
            """
            Use inversion method to simulate and return the initial state and corresponding action,
            given some fixed policy
            
            Parameters: the current state the action
            
            Return: the initial state and action
            """
            # uniformly generate a state
            n = len(self.states)
            u1 = np.random.uniform(0,1)
            for i in range(n):
                if i*1.0/n < u1 <= (i+1)*1.0/n: 
                    s = self.states[i]
                    break
            # sample the corresponding action given some fixed policy
            u2 = np.random.uniform(0,1)
            cum_proba = 0
            for action,proba in policy_dict[s].items():
                cum_proba += proba
                if u2 <= cum_proba:
                    a = action
                    break 
            return s,a
        
        def simulate_transition(self,s,a):
            """
            Use inversion method to simulate and return the next state and the corresponding reward
            
            Parameters: the current state the action
            
            Return: the next sample state and reward
            """
            u = np.random.uniform(0,1)
            proba_dict = self.transition_proba[s][a]
            cum_proba = 0
            for i in range(len(self.states)):
                cum_proba += proba_dict[self.states[i]]
                if u <= cum_proba:
                    s_new = self.states[i]
                    break  
            r = self.reward[s][a][s_new]
            return s_new,r

In [11]:
my_mdp = MarkovDecisionProcess(T,R,1,1)   # no discount, tol=1

In [12]:
optimal_policy = my_mdp.policy_iteration()
optimal_policy

{(0, 0): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (0, 1): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (0, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (0, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 0): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 1): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (2, 0): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 1): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (3, 0): {'Up': 0, 'Right': 0, 'Down': 0, 'Left': 1},
 (3, 1): {'Up': 0, 'Right': 0, 'Down': 0, 'Left': 1},
 (3, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (3, 3): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0}}

## 4. Model-Free Prediction

We used the original Markov Decision Process under the optimal policy to generate episodes of observations (R,S,A), and estimate the value functions using Monte Carlo learning and Temporal Difference Learning methods. 

The value function given by the original MDP under the optimal policy is

In [13]:
toy_policy = optimal_policy
mrp_pre = my_mdp.to_MRP(toy_policy)
mdp_value_function = mrp_pre.get_value_function()
mdp_value_function

{(0, 0): 1.71009019838139,
 (0, 1): 3.071932119775163,
 (0, 2): 4.449914383561644,
 (0, 3): 5.699914383561644,
 (1, 0): 0.8153548272312068,
 (1, 1): 2.0480740094833125,
 (1, 3): 7.106164383561644,
 (2, 0): 1.9112649294496935,
 (2, 1): 3.4521639072648256,
 (2, 2): 5.205479452054796,
 (2, 3): 8.356164383561644,
 (3, 0): 0.679983209147123,
 (3, 1): 0.8297294467265717,
 (3, 2): -10.0,
 (3, 3): 10.0}

### 4.1 Monte Carlo Learning 

In [14]:
def Monte_Carlo_learning(mdp,policy_dict,k):  
    """
    The MC learning algorithm predicts the value functions of an MDP object given some fixed policy, after 
    k episodes
    
    Parameters: 
        mdp object: representing the environment the algorithm interacts with
        policy_dict: a fixed policy to evaluate
        k: the number of episodes
        
    Return: value functions
    """
    n = len(mdp.states)
    value_dict={}
    count_dict={}
    for state in mdp.states:
        value_dict[state] = 0
        count_dict[state] = 0
            
    # simulate k episodes
    for j in range(k):
        # simulate the initial state
        s,a = mdp.generate_initial_state_action(policy_dict)
        episode= [(0,s,a)]  
        
        # simulate the following transitions until hitting the terminal state
        while not mdp.is_terminal(s,a):    # while the state is not self absorbing, i.e. is not the terminal state
            s, r = mdp.simulate_transition(s,a)
            u2 = np.random.uniform(0,1)
            cum_proba = 0
            for action,proba in policy_dict[s].items():
                cum_proba += proba
                if u2 <= cum_proba:
                    a = action
                    break 
            episode.append((r,s,a))

        # update value function for each state that appears in the episode
        for state in mdp.states:
            t = -1
            for j in range(len(episode)):
                if episode[j][1] == state: 
                    t = j
                    break
            if t >= 0: # if the state appears in the current episode, we update its value function
                if t == len(episode)-1:       # the terminal state
                    G = 10 if episode[t][1] == (3,3) else -10
                else:
                    G = 0
                    for j in range(t+1,len(episode)): G += episode[j][0]*mdp.gamma**(j-1-t)
                    end_reward = 10 if episode[len(episode)-1][1] == (3,3) else -10
                    G += end_reward*mdp.gamma**(len(episode)-1-t)
                count_dict[state] += 1
                value_dict[state] += 1.0/count_dict[state]*(G - value_dict[state])          
        
    return value_dict

The predicted value functions by MC after 100,000 episodes is

In [17]:
Monte_Carlo_learning(my_mdp,toy_policy,100000)

{(0, 0): 1.6344118085582204,
 (0, 1): 2.989469485440768,
 (0, 2): 4.393167499456858,
 (0, 3): 5.646228730366502,
 (1, 0): 0.7729044834308006,
 (1, 1): 2.0664186623265777,
 (1, 3): 7.064985669661557,
 (2, 0): 1.925476358503885,
 (2, 1): 3.46287265476734,
 (2, 2): 5.191906405830398,
 (2, 3): 8.338632254846726,
 (3, 0): 0.6835730507191533,
 (3, 1): 0.9828378261246438,
 (3, 2): -10.0,
 (3, 3): 10.0}

### 4.2 Forward TD(0) Learning

In [18]:
def Temporal_Difference_learning(mdp,policy_dict,alpha,lambda_TD,n):
    """
    Use TD(lambda) to predict the value functions of a given policy
    
    Parameters: 
        mdp object
        given policy dictionary
        step size alpha
        lambda_TD
        the minimum number of iterations n
        
    Return:
        value function dictionary
    """
    count = 0
    value_dict = {}
    # randomly initialize the value function following a standard normal distribution
    for state in mdp.states: value_dict[state] = np.random.randn()+3
    # terminal value
    value_dict[(3,3)] = 10
    value_dict[(3,2)] = -10
    
    # n controls the smallest number of observations to get a good estimation
    while count < n:
        count += 1
        s,a = mdp.generate_initial_state_action(policy_dict)
        while not mdp.is_terminal(s,a):    # while the state is not self absorbing, i.e. is not the terminal state
            count += 1
            s_next, r = mdp.simulate_transition(s,a)
            TD_target = r + mdp.gamma*value_dict[s_next] - value_dict[s]
            value_dict[s] += alpha*TD_target
            s = s_next
            u2 = np.random.uniform(0,1)
            cum_proba = 0
            for action,proba in policy_dict[s].items():
                cum_proba += proba
                if u2 <= cum_proba:
                    a = action
                    break 
    return value_dict

Given step size alpha = 0.001, the predicted value functions by forward TD(0) after 100,000 time steps (results are unstable when the the number of iterations is small) is

In [20]:
Temporal_Difference_learning(my_mdp,toy_policy,0.001,0,100000)

{(0, 0): 1.0238378127503425,
 (0, 1): 1.8796396802413093,
 (0, 2): 3.7855300979426976,
 (0, 3): 5.503489729177209,
 (1, 0): 1.1141168913608221,
 (1, 1): 2.1086685456157688,
 (1, 3): 7.1214770811154,
 (2, 0): 2.066971194715635,
 (2, 1): 3.578692426924436,
 (2, 2): 5.246023287076667,
 (2, 3): 8.351350796532826,
 (3, 0): 1.0093431114984481,
 (3, 1): 0.9795103563825974,
 (3, 2): -10,
 (3, 3): 10}

The predicted values are highly sensitive to the initial random initialization, even after 10000 time steps. Thus MC learning algorithm is better w.r.t. this MDP problem.

## 5. Model-Free Control 

### 5.1 Sarsa 

In [21]:
def epsilon_greedy_policy_improvement(mdp,action_value_dict,l,epsilon):
    """
    A helper function that return the epsilon-greedy policy dictionary, given all the action values of some state
    """
    state_epsilon_greedy_policy = {}
    max_action_value = -10000
    max_action = ""
    for action in mdp.actions:
        state_epsilon_greedy_policy[action] = epsilon/l
        if action_value_dict[action] > max_action_value:
            max_action_value = action_value_dict[action]
            max_action = action
    state_epsilon_greedy_policy[max_action] += 1-epsilon
    return state_epsilon_greedy_policy

In [22]:
def Sarsa(mdp,alpha,lambda_TD,n,epsilon):
    """
    Use Sarsa algorithm to solve the optimal value function, and derive the corresponding optimal policy
    
    Parameters:
        mdp object
        step size alpha
        lambda TD
        the minimum number of iterations n
        epsilon
    
    Return:
        optimal value dictionary
        epsilon greedy policy dictionary
    """
    count = 0
    state_action_value_dict = {}
    epsilon_greedy_policy = {}
    m = len(mdp.states)
    l = len(mdp.actions)
      
    for state in mdp.states:
        action_value_dict = {}
        for action in mdp.actions:
            # randomly initialize the value function following a standard normal distribution
            if state!= (3,3) and state!=(3,2):
                action_value_dict[action] = np.random.randn()+3
            elif state == (3,3):
                action_value_dict[action] = 10
            else:
                action_value_dict[action] = -10
        # apply the epsilon-greedy policy improvement to the initial Q function 
        epsilon_greedy_policy[state] = epsilon_greedy_policy_improvement(mdp,action_value_dict,l,epsilon)
        state_action_value_dict[state] = action_value_dict
        
    while count < n:
        count += 1
        s,a = mdp.generate_initial_state_action(epsilon_greedy_policy)
        while not mdp.is_terminal(s,a):    # while the state is not self absorbing, i.e. is not the terminal state
            count += 1
            s_next, r = mdp.simulate_transition(s,a)
            # sample A' using the current epsilon-greedy policy
            u1 = np.random.uniform(0,1)
            cum_proba = 0
            for action in mdp.actions:
                cum_proba += epsilon_greedy_policy[s_next][action]
                if u1 <= cum_proba:
                    a_next = action
                    break
            # update Q(S,A)
            state_action_value_dict[s][a] += alpha*(r+mdp.gamma*state_action_value_dict[s_next][a_next] - state_action_value_dict[s][a])
            # apply the epsilon-greedy policy improvement
            epsilon_greedy_policy[s] = epsilon_greedy_policy_improvement(mdp,state_action_value_dict[s],l,epsilon)
            s = s_next
            a = a_next
    
    # finally we extract the optimal action-value function for each state
    state_value_dict = {}
    for state in mdp.states:
        max_action_value = -10000
        for action in mdp.actions:
            if state_action_value_dict[state][action]>max_action_value:
                max_action_value = state_action_value_dict[state][action]
        state_value_dict[state] = max_action_value
    return state_value_dict, epsilon_greedy_policy

In [25]:
optimal_action_value_function, optimal_policy_sarsa = Sarsa(my_mdp,0.01,0,100000,0.001)
optimal_action_value_function

{(0, 0): 1.6290125893626852,
 (0, 1): 3.105362444045345,
 (0, 2): 4.518068305020353,
 (0, 3): 5.752159212775516,
 (1, 0): 1.2007893310893003,
 (1, 1): 2.34575790707091,
 (1, 3): 7.1372594549448936,
 (2, 0): 2.179383044661068,
 (2, 1): 3.845989734765322,
 (2, 2): 5.600212160187006,
 (2, 3): 8.389643128062108,
 (3, 0): 0.8702483710270821,
 (3, 1): 0.9091863218607308,
 (3, 2): -10,
 (3, 3): 10}

The benchmark MDP value function is

In [224]:
mdp_value_function

{(0, 0): 1.71009019838139,
 (0, 1): 3.071932119775163,
 (0, 2): 4.449914383561644,
 (0, 3): 5.699914383561644,
 (1, 0): 0.8153548272312068,
 (1, 1): 2.0480740094833125,
 (1, 3): 7.106164383561644,
 (2, 0): 1.9112649294496935,
 (2, 1): 3.4521639072648256,
 (2, 2): 5.205479452054796,
 (2, 3): 8.356164383561644,
 (3, 0): 0.679983209147123,
 (3, 1): 0.8297294467265717,
 (3, 2): -10.0,
 (3, 3): 10.0}

The optimal action-value function is in general similar to the value iteration results of the underlying MDP. However, the derived optimal policy is slightly different than the MDP solution for 1/2 states

In [26]:
optimal_policy_sarsa

{(0, 0): {'Up': 0.99925, 'Right': 0.00025, 'Down': 0.00025, 'Left': 0.00025},
 (0, 1): {'Up': 0.99925, 'Right': 0.00025, 'Down': 0.00025, 'Left': 0.00025},
 (0, 2): {'Up': 0.99925, 'Right': 0.00025, 'Down': 0.00025, 'Left': 0.00025},
 (0, 3): {'Up': 0.00025, 'Right': 0.99925, 'Down': 0.00025, 'Left': 0.00025},
 (1, 0): {'Up': 0.99925, 'Right': 0.00025, 'Down': 0.00025, 'Left': 0.00025},
 (1, 1): {'Up': 0.00025, 'Right': 0.99925, 'Down': 0.00025, 'Left': 0.00025},
 (1, 3): {'Up': 0.00025, 'Right': 0.99925, 'Down': 0.00025, 'Left': 0.00025},
 (2, 0): {'Up': 0.99925, 'Right': 0.00025, 'Down': 0.00025, 'Left': 0.00025},
 (2, 1): {'Up': 0.99925, 'Right': 0.00025, 'Down': 0.00025, 'Left': 0.00025},
 (2, 2): {'Up': 0.99925, 'Right': 0.00025, 'Down': 0.00025, 'Left': 0.00025},
 (2, 3): {'Up': 0.00025, 'Right': 0.99925, 'Down': 0.00025, 'Left': 0.00025},
 (3, 0): {'Up': 0.00025, 'Right': 0.00025, 'Down': 0.00025, 'Left': 0.99925},
 (3, 1): {'Up': 0.00025, 'Right': 0.00025, 'Down': 0.00025, 'Lef

The benchmark optimal policy of the underlying MDP is

In [233]:
optimal_policy

{(0, 0): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (0, 1): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (0, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (0, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 0): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 1): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (2, 0): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 1): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (3, 0): {'Up': 0, 'Right': 0, 'Down': 0, 'Left': 1},
 (3, 1): {'Up': 0, 'Right': 0, 'Down': 0, 'Left': 1},
 (3, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (3, 3): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0}}

### 5.2 Q-Learning 

In this section, we would like to learn the optimal policy while following an epsilon-greedy policy

In [27]:
def Q_learning(mdp,alpha,lambda_TD,n,epsilon):
    """
    Use Q-learning to learn the optimal value function
    
    Parameters:
        mdp object
        step size alpha
        lambda TD
        the minimum number of iterations n
        epsilon
    
    Return:
        optimal value function dictionary
    """
    count = 0
    state_action_value_dict = {}
    epsilon_greedy_policy = {}
    m = len(mdp.states)
    l = len(mdp.actions)
    
    # we use the initial epsilon-greedy policy as the behavior policy
    for state in mdp.states:
        action_value_dict = {}
        for action in mdp.actions:
            # randomly initialize the value function following a standard normal distribution
            if state!= (3,3) and state!=(3,2):
                action_value_dict[action] = np.random.randn()+3
            elif state == (3,3):
                action_value_dict[action] = 10
            else:
                action_value_dict[action] = -10
        # apply the epsilon-greedy policy improvement to the initial Q function 
        epsilon_greedy_policy[state] = epsilon_greedy_policy_improvement(mdp,action_value_dict,l,epsilon)
        state_action_value_dict[state] = action_value_dict
        
    while count < n:
        count += 1
        s,a = mdp.generate_initial_state_action(epsilon_greedy_policy)
        while not mdp.is_terminal(s,a):    # while the state is not self absorbing, i.e. is not the terminal state
            count += 1
            s_next, r = mdp.simulate_transition(s,a)
            max_action_value = -10000
            for action in mdp.actions:
                if state_action_value_dict[s_next][action]>max_action_value: max_action_value = state_action_value_dict[s_next][action]
            # update Q(S,A) following the greedy policy pi
            state_action_value_dict[s][a] += alpha*(r+mdp.gamma*max_action_value - state_action_value_dict[s][a])
            # apply the epsilon-greedy policy improvement
            epsilon_greedy_policy[s] = epsilon_greedy_policy_improvement(mdp,state_action_value_dict[s],l,epsilon)
            s = s_next
            # sample A using the epsilon-greedy policy
            u1 = np.random.uniform(0,1)
            cum_proba = 0
            for action in mdp.actions:
                cum_proba += epsilon_greedy_policy[s][action]
                if u1 <= cum_proba:
                    a = action
                    break

    # finally we extract the optimal action-value function for each state
    state_value_dict = {}
    for state in mdp.states:
        max_action_value = -10000
        for action in mdp.actions:
            if state_action_value_dict[state][action]>max_action_value:
                max_action_value = state_action_value_dict[state][action]
        state_value_dict[state] = max_action_value
    return state_value_dict      

In [28]:
Q_learning(my_mdp,0.01,0,100000,0.001)

{(0, 0): 1.6895757849754014,
 (0, 1): 3.1479838437774355,
 (0, 2): 4.433779538588049,
 (0, 3): 5.580674923400579,
 (1, 0): 1.1545773060535598,
 (1, 1): 2.0624480291476215,
 (1, 3): 7.238937574808567,
 (2, 0): 2.1223488419520122,
 (2, 1): 3.34380762845725,
 (2, 2): 4.924513303626981,
 (2, 3): 8.596780930433304,
 (3, 0): 1.007272783043343,
 (3, 1): 1.2570082465835943,
 (3, 2): -10,
 (3, 3): 10}

The benchmark MDP value function is

In [223]:
mdp_value_function

{(0, 0): 1.71009019838139,
 (0, 1): 3.071932119775163,
 (0, 2): 4.449914383561644,
 (0, 3): 5.699914383561644,
 (1, 0): 0.8153548272312068,
 (1, 1): 2.0480740094833125,
 (1, 3): 7.106164383561644,
 (2, 0): 1.9112649294496935,
 (2, 1): 3.4521639072648256,
 (2, 2): 5.205479452054796,
 (2, 3): 8.356164383561644,
 (3, 0): 0.679983209147123,
 (3, 1): 0.8297294467265717,
 (3, 2): -10.0,
 (3, 3): 10.0}

## 6. Value function Approximation 

With regard to my previous MDP problem, I explore using a linear model to approximate the value functions. To start with a simple model, I use a naive feature representation of the state space, i.e. an one-hot-vector. Then I build a linear model to find the optimal weights and the corresponding action-value functions.

### 6.1 Monte Carlo prediction 

In [39]:
# featue representation of states
feature_state_matrix = np.eye(len(my_mdp.states),len(my_mdp.states))
feature_state = {}
i = 0
for state in my_mdp.states:
    feature_state[state] = feature_state_matrix[:,i]
    i += 1
feature_state

{(0, 0): array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 (0, 1): array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 (0, 2): array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 (0, 3): array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 (1, 0): array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 (1, 1): array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 (1, 3): array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 (2, 0): array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 (2, 1): array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 (2, 2): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 (2, 3): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 (3, 0): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 (3, 1): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [60]:
def Monte_Carlo_value_func_approximation(mdp,feature_rep,policy_dict,k,alpha):
    # random initialization of weights
    n = len(mdp.states)
    w = np.random.randn(n)
      
    # simulate k episodes    
    for j in range(k):
        # simulate the initial state
        s,a = mdp.generate_initial_state_action(policy_dict)
        episode= [(0,s,a)]  
        
        # simulate the following transitions until hitting the terminal state
        while not mdp.is_terminal(s,a):    # while the state is not self absorbing, i.e. is not the terminal state
            s, r = mdp.simulate_transition(s,a)
            u2 = np.random.uniform(0,1)
            cum_proba = 0
            for action,proba in policy_dict[s].items():
                cum_proba += proba
                if u2 <= cum_proba:
                    a = action
                    break 
            episode.append((r,s,a))
        
        # prepare the "training data" (S_t, G_t)
        T = len(episode)
        G_vec = np.zeros(T)
        for t in range(T):
            G = 0
            if t < T-1:
                for j in range(t+1,T): G += episode[j][0]*mdp.gamma**(j-1-t)
            end_reward = 10 if episode[T-1][1] == (3,3) else -10
            G += end_reward*mdp.gamma**(T-1-t) 
            G_vec[t] = G
        
        # stochastic gradient descent     
        perm = np.random.permutation(T)
        for j in perm:
            st = episode[j][1]
            w += alpha*(G_vec[j] - np.dot(feature_rep[st],w))*feature_rep[st]
    
    # return the predicted value functions
    value_dict = {}
    for state in mdp.states:
        value_dict[state] = np.dot(feature_rep[state],w)
    
    return value_dict

In [62]:
Monte_Carlo_value_func_approximation(my_mdp,feature_state,toy_policy,100000,0.001)

{(0, 0): 1.5429151180014966,
 (0, 1): 3.003894839330005,
 (0, 2): 4.42200389599265,
 (0, 3): 5.731061326558239,
 (1, 0): 0.9696373702623957,
 (1, 1): 2.036426482014669,
 (1, 3): 7.199549233208328,
 (2, 0): 2.151298167513606,
 (2, 1): 3.3807116065603946,
 (2, 2): 5.124856998954625,
 (2, 3): 8.359090140023216,
 (3, 0): 0.7061300664306106,
 (3, 1): 1.015939499400649,
 (3, 2): -9.999994267863787,
 (3, 3): 9.999999999999112}

The bench mark MDP optimal value function is

In [63]:
mdp_value_function

{(0, 0): 1.71009019838139,
 (0, 1): 3.071932119775163,
 (0, 2): 4.449914383561644,
 (0, 3): 5.699914383561644,
 (1, 0): 0.8153548272312068,
 (1, 1): 2.0480740094833125,
 (1, 3): 7.106164383561644,
 (2, 0): 1.9112649294496935,
 (2, 1): 3.4521639072648256,
 (2, 2): 5.205479452054796,
 (2, 3): 8.356164383561644,
 (3, 0): 0.679983209147123,
 (3, 1): 0.8297294467265717,
 (3, 2): -10.0,
 (3, 3): 10.0}

The predicted value functions by the MC algorithm is very similar to the MDP solution.

### 6.2 TD(0) prediction 

In [69]:
def Temporal_Difference_value_func_approximation(mdp,feature_rep,policy_dict,alpha,lambda_TD,n):
    # random initialization of weights
    w = np.random.randn(len(mdp.states))
    
    # n controls the smallest number of observations to get a good estimation
    count = 0
    while count < n:
        # sample an episode
        count += 1
        s,a = mdp.generate_initial_state_action(policy_dict)
        episode = []
        while not mdp.is_terminal(s,a):    # while the state is not self absorbing, i.e. is not the terminal state
            count += 1
            s_next, r = mdp.simulate_transition(s,a)
            episode.append((r,s,s_next))
            s = s_next
            u2 = np.random.uniform(0,1)
            cum_proba = 0
            for action,proba in policy_dict[s].items():
                cum_proba += proba
                if u2 <= cum_proba:
                    a = action
                    break     
        # stochastic gradient descent for data in the episode   
        T = len(episode)
        perm = np.random.permutation(T)
        for j in perm:
            s = episode[j][1]
            s_next = episode[j][2]
            r = episode[j][0]
            if j == T-1:
                next_value = 10 if s_next == (3,3) else -10
            else:
                next_value = np.dot(feature_rep[s_next],w)
            TD_target = r + mdp.gamma*next_value - np.dot(feature_rep[s],w)                
            w += alpha*TD_target*feature_rep[s]
    
    # return the predicted value functions
    value_dict = {}
    for state in mdp.states:
        value_dict[state] = np.dot(feature_rep[state],w)
    value_dict[(3,2)] = -10
    value_dict[(3,3)] = 10
    return value_dict

In [73]:
Temporal_Difference_value_func_approximation(my_mdp,feature_state,toy_policy,0.001,0,1000000)

{(0, 0): 1.7793095235761278,
 (0, 1): 3.0698271210640735,
 (0, 2): 4.461107635328946,
 (0, 3): 5.695309314930527,
 (1, 0): 0.7964916915753437,
 (1, 1): 1.9234302587006789,
 (1, 3): 7.063135039390187,
 (2, 0): 1.7922130535537446,
 (2, 1): 3.258505851343289,
 (2, 2): 4.971389016136814,
 (2, 3): 8.36150595599048,
 (3, 0): 0.6862559976098094,
 (3, 1): 0.830344174962392,
 (3, 2): -10,
 (3, 3): 10}

The bench mark MDP optimal value function is

In [72]:
mdp_value_function

{(0, 0): 1.71009019838139,
 (0, 1): 3.071932119775163,
 (0, 2): 4.449914383561644,
 (0, 3): 5.699914383561644,
 (1, 0): 0.8153548272312068,
 (1, 1): 2.0480740094833125,
 (1, 3): 7.106164383561644,
 (2, 0): 1.9112649294496935,
 (2, 1): 3.4521639072648256,
 (2, 2): 5.205479452054796,
 (2, 3): 8.356164383561644,
 (3, 0): 0.679983209147123,
 (3, 1): 0.8297294467265717,
 (3, 2): -10.0,
 (3, 3): 10.0}

We can see that the TD(0)-based value approximation converges to the MDP optimal solution after 1,000,000 time steps. The algorithm yields unstable results after 100,000 time steps, which guarantees convergence of the tabular TD(0) algorithm. Therefore, it takes more time for the TD(0)-based value approximation algorithm to converge.

### 6.3 Sarsa control 

In [74]:
feature_state_action_matrix = np.eye(len(my_mdp.states)*len(my_mdp.actions))
feature_state_action = {}
i = 0
for state in my_mdp.states:
    feature_action = {}
    for action in my_mdp.actions:
        feature_action[action] = feature_state_action_matrix[:,i]
        i += 1
    feature_state_action[state] = feature_action
feature_state_action

{(0,
  0): {'Up': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Right': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Down': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Left': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [85]:
def Sarsa_value_func_approximation(mdp,feature_rep,alpha,lambda_TD,n,epsilon):
    count = 0    
    state_action_value_dict = {}
    epsilon_greedy_policy = {}
    m = len(mdp.states)
    l = len(mdp.actions)
    w = np.random.randn(m*l)
      
    for state in mdp.states:
        action_value_dict = {}
        for action in mdp.actions:
            if state == (3,3):
                action_value_dict[action] = 10
            elif state == (3,2):
                action_value_dict[action] = -10
            else:
                action_value_dict[action] = np.dot(feature_rep[state][action],w)
        epsilon_greedy_policy[state] = epsilon_greedy_policy_improvement(mdp,action_value_dict,l,epsilon)
        state_action_value_dict[state] = action_value_dict
        
    while count < n:
        count += 1
        s,a = mdp.generate_initial_state_action(epsilon_greedy_policy)
        while not mdp.is_terminal(s,a):    # while the state is not self absorbing, i.e. is not the terminal state
            count += 1
            s_next, r = mdp.simulate_transition(s,a)
            # sample A' using the current epsilon-greedy policy
            u1 = np.random.uniform(0,1)
            cum_proba = 0
            for action in mdp.actions:
                cum_proba += epsilon_greedy_policy[s_next][action]
                if u1 <= cum_proba:
                    a_next = action
                    break
            # update w
            if mdp.is_terminal(s_next,a_next):
                next_value = 10 if s_next == (3,3) else -10
            else: 
                next_value = np.dot(feature_rep[s_next][a_next],w)
            w += alpha*(r+mdp.gamma*next_value - np.dot(feature_rep[s][a],w))*feature_rep[s][a]
            
            # update Q(s,a) and apply the epsilon-greedy policy improvement
            for state in mdp.states:
                action_value_dict = {}
                for action in mdp.actions:
                    if state == (3,3):
                        action_value_dict[action] = 10
                    elif state == (3,2):
                        action_value_dict[action] = -10
                    else:
                        action_value_dict[action] = np.dot(feature_rep[state][action],w)
                epsilon_greedy_policy[state] = epsilon_greedy_policy_improvement(mdp,action_value_dict,l,epsilon)
                state_action_value_dict[state] = action_value_dict
            s = s_next
            a = a_next
    
    # finally we extract the optimal action-value function for each state
    state_value_dict = {}
    for state in mdp.states:
        max_action_value = -10000
        for action in mdp.actions:
            if state_action_value_dict[state][action]>max_action_value:
                max_action_value = state_action_value_dict[state][action]
        state_value_dict[state] = max_action_value
        
    return state_value_dict, epsilon_greedy_policy

In [88]:
state_value_dict_sarsa_func, greedy_policy_sarsa_func = Sarsa_value_func_approximation(my_mdp,feature_state_action,0.001,0,1000000,0.01)

In [89]:
state_value_dict_sarsa_func

{(0, 0): -0.6087862053619488,
 (0, 1): 0.7366269402309574,
 (0, 2): 4.17471518230005,
 (0, 3): 5.528982427311625,
 (1, 0): 0.5086764723772177,
 (1, 1): 1.7353188274122966,
 (1, 3): 6.99832751323632,
 (2, 0): 1.6156745265934085,
 (2, 1): 3.209931648439985,
 (2, 2): 5.0487797712236215,
 (2, 3): 8.367138593020785,
 (3, 0): 0.3588144875045164,
 (3, 1): 0.600767770421827,
 (3, 2): -10,
 (3, 3): 10}

The bench mark MDP optimal value function is

In [94]:
mdp_value_function

{(0, 0): 1.71009019838139,
 (0, 1): 3.071932119775163,
 (0, 2): 4.449914383561644,
 (0, 3): 5.699914383561644,
 (1, 0): 0.8153548272312068,
 (1, 1): 2.0480740094833125,
 (1, 3): 7.106164383561644,
 (2, 0): 1.9112649294496935,
 (2, 1): 3.4521639072648256,
 (2, 2): 5.205479452054796,
 (2, 3): 8.356164383561644,
 (3, 0): 0.679983209147123,
 (3, 1): 0.8297294467265717,
 (3, 2): -10.0,
 (3, 3): 10.0}

The result is less accurate than the non-parametric sarsa(0) algorithm even we iterate for a longer time.

In [93]:
greedy_policy_sarsa_func_final = {}
for state in my_mdp.states:
    greedy = {}
    for action in my_mdp.actions:
        greedy[action] = round(greedy_policy_sarsa_func[state][action])
    greedy_policy_sarsa_func_final[state] = greedy
greedy_policy_sarsa_func_final

{(0, 0): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (0, 1): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (0, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (0, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 0): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (1, 1): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (2, 0): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 1): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (3, 0): {'Up': 0, 'Right': 0, 'Down': 0, 'Left': 1},
 (3, 1): {'Up': 0, 'Right': 0, 'Down': 0, 'Left': 1},
 (3, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (3, 3): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0}}

The benchmark MDP optimal policy is 

In [95]:
optimal_policy

{(0, 0): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (0, 1): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (0, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (0, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 0): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 1): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (1, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (2, 0): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 1): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (2, 3): {'Up': 0, 'Right': 1, 'Down': 0, 'Left': 0},
 (3, 0): {'Up': 0, 'Right': 0, 'Down': 0, 'Left': 1},
 (3, 1): {'Up': 0, 'Right': 0, 'Down': 0, 'Left': 1},
 (3, 2): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0},
 (3, 3): {'Up': 1, 'Right': 0, 'Down': 0, 'Left': 0}}

The optimal policy is less accurate as well.

### 6.4 Q-Learning control 

In [96]:
def Q_learning_func_approximation(mdp,feature_rep,alpha,lambda_TD,n,epsilon):
    count = 0    
    state_action_value_dict = {}
    epsilon_greedy_policy = {}
    m = len(mdp.states)
    l = len(mdp.actions)
    w = np.random.randn(m*l)
      
    for state in mdp.states:
        action_value_dict = {}
        for action in mdp.actions:
            if state == (3,3):
                action_value_dict[action] = 10
            elif state == (3,2):
                action_value_dict[action] = -10
            else:
                action_value_dict[action] = np.dot(feature_rep[state][action],w)
        epsilon_greedy_policy[state] = epsilon_greedy_policy_improvement(mdp,action_value_dict,l,epsilon)
        state_action_value_dict[state] = action_value_dict
        
    while count < n:
        count += 1
        s,a = mdp.generate_initial_state_action(epsilon_greedy_policy)
        while not mdp.is_terminal(s,a):    # while the state is not self absorbing, i.e. is not the terminal state
            count += 1
            s_next, r = mdp.simulate_transition(s,a)
            max_action_value = -10000
            for action in mdp.actions:
                if state_action_value_dict[s_next][action]>max_action_value: max_action_value = state_action_value_dict[s_next][action]
            # update Q(S,A) following the greedy policy pi
            w += alpha*(r+mdp.gamma*max_action_value - np.dot(feature_rep[s][a],w))*feature_rep[s][a]
            # update Q(s,a) and apply the epsilon-greedy policy improvement
            for state in mdp.states:
                action_value_dict = {}
                for action in mdp.actions:
                    if state == (3,3):
                        action_value_dict[action] = 10
                    elif state == (3,2):
                        action_value_dict[action] = -10
                    else:
                        action_value_dict[action] = np.dot(feature_rep[state][action],w)
                epsilon_greedy_policy[state] = epsilon_greedy_policy_improvement(mdp,action_value_dict,l,epsilon)
                state_action_value_dict[state] = action_value_dict
            s = s_next
            # sample A using the epsilon-greedy policy
            u1 = np.random.uniform(0,1)
            cum_proba = 0
            for action in mdp.actions:
                cum_proba += epsilon_greedy_policy[s][action]
                if u1 <= cum_proba:
                    a = action
                    break

    # finally we extract the optimal action-value function for each state
    state_value_dict = {}
    for state in mdp.states:
        max_action_value = -10000
        for action in mdp.actions:
            if state_action_value_dict[state][action]>max_action_value:
                max_action_value = state_action_value_dict[state][action]
        state_value_dict[state] = max_action_value
    return state_value_dict      

In [97]:
Q_learning_func_approximation(my_mdp,feature_state_action,0.001,0,100000,0.01)

{(0, 0): -2.3216445075736973,
 (0, 1): -1.756378166046778,
 (0, 2): -0.10396899756069912,
 (0, 3): 2.2819146078754713,
 (1, 0): -2.5451031119913825,
 (1, 1): -2.3505910595643833,
 (1, 3): 5.429967900250688,
 (2, 0): -2.5258899864261273,
 (2, 1): -0.885652515799276,
 (2, 2): 2.960619580835856,
 (2, 3): 7.8850191263841936,
 (3, 0): -2.883300550315505,
 (3, 1): -2.966706117288369,
 (3, 2): -10,
 (3, 3): 10}

The Q-learning algorithm diverges with value approximation.

## 7. Policy Gradient - REINFORCE Algorithm

In [103]:
def Monte_Carlo_policy_gradient(mdp,feature_action_rep,k,alpha):
    # random initialization of weights
    theta = np.random.uniform(0,1,len(mdp.actions)*len(mdp.states))
    policy_dict = {}
    for state in mdp.states:
        policy_dict_state = {}
        normalization = 0
        for action in mdp.actions:
            proba_raw = np.dot(feature_action_rep[state][action],theta)
            policy_dict_state[action] = proba_raw
            normalization += proba_raw
        for action in mdp.actions:
            policy_dict_state[action] /= normalization
        policy_dict[state] = policy_dict_state
    
    # simulate k episodes    
    for j in range(k):
        # simulate the initial state
        s,a = mdp.generate_initial_state_action(policy_dict)
        episode= [(0,s,a)]  
        
        # simulate the following transitions until hitting the terminal state
        while not mdp.is_terminal(s,a):    # while the state is not self absorbing, i.e. is not the terminal state
            s, r = mdp.simulate_transition(s,a)
            u2 = np.random.uniform(0,1)
            cum_proba = 0
            for action,proba in policy_dict[s].items():
                cum_proba += proba
                if u2 <= cum_proba:
                    a = action
                    break 
            episode.append((r,s,a))
        
        # prepare the "training data" (S_t, G_t)
        T = len(episode)
        G_vec = np.zeros(T)
        for t in range(T):
            G = 0
            if t < T-1:
                for j in range(t+1,T): G += episode[j][0]*mdp.gamma**(j-1-t)
            end_reward = 10 if episode[T-1][1] == (3,3) else -10
            G += end_reward*mdp.gamma**(T-1-t) 
            G_vec[t] = G
        
        # stochastic gradient descent     
        perm = np.random.permutation(T)
        for j in perm:
            st = episode[j][1]
            at = episode[j][2]
            theta += alpha*G_vec[j]*feature_action_rep[st][at]/np.dot(feature_action_rep[st][at],theta)
            for state in mdp.states:
                policy_dict_state = {}
                normalization = 0
                for action in mdp.actions:
                    proba_raw = np.dot(feature_action_rep[state][action],theta)
                    policy_dict_state[action] = proba_raw
                    normalization += proba_raw
                for action in mdp.actions:
                    policy_dict_state[action] /= normalization
                policy_dict[state] = policy_dict_state
    
    
    return policy_dict

In [104]:
Monte_Carlo_policy_gradient(my_mdp,feature_state_action,1000000,0.001)

{(0, 0): {'Up': 1.0002954143121976,
  'Right': 0.2700764388135012,
  'Down': -0.8441483823324725,
  'Left': 0.5737765292067736},
 (0, 1): {'Up': 0.7027802621965207,
  'Right': 0.24176202983028575,
  'Down': 0.0715890455648956,
  'Left': -0.016131337591701937},
 (0, 2): {'Up': 1.2122995498136961,
  'Right': -0.2129654324977476,
  'Down': -0.021286676266119103,
  'Left': 0.02195255895017046},
 (0, 3): {'Up': -0.9029063736885005,
  'Right': 2.979118665104538,
  'Down': 0.003918619685761708,
  'Left': -1.0801309111017992},
 (1, 0): {'Up': -0.03980072681119881,
  'Right': 1.039545085559207,
  'Down': 3.617435078809529,
  'Left': -3.6171794375575375},
 (1, 1): {'Up': 1.0000488662111877,
  'Right': 0.009736053378603945,
  'Down': 0.08783281383056121,
  'Left': -0.09761773342035297},
 (1, 3): {'Up': -0.0016685400234855464,
  'Right': 1.0101497919848068,
  'Down': 0.005862622717644371,
  'Left': -0.014343874678965701},
 (2, 0): {'Up': 1.006530605245872,
  'Right': 0.03250839832053037,
  'Down':

We can see the algorithm diverges even after 1,000,000 iterations since some of the (normalized) probabilities are negative. One potential reason is that the action space is too small.